## **ALGORITMO GENÉTICO - CARLOS MERCADAL**

El objetivo del algoritmo es obtener la cartera óptima para los 70.000 fondos de inversión disponibles. Para ello vamos a utilizar el set de datos ya limpios y homogenizados que utilizamos para el algoritmo de los monos.

**1. Los inversores (cromosomas) deben poder generar carteras con un número variable de activos. Estableceremos el mínimo en 1 activo y 20 en el máximo. El número de activos debe poder modificarse entre generaciones, sin caer en extremos (que siempre salgan carteras de 1 activo o de 20). Debemos pensar un sistema coherente, para resolver este punto.**

En primer lugar es necesario determinar el número de individuos que van a formar parte de la "primera generación", y determinar qué numero de generaciones posteriores queremos utilizar. En este caso hemos realizado 50 individuos y 50 generaciones. La primera generación va a generarse de forma aleatoria escogiendo activos aleatorios. Las carteras van a estar compuesta por un número aleatorio de activos dentro del rango mínimo y máximo, de forma que acabemos teniendo 50 carteras de tamaños distintos y activos distintos. Para cada individuo, posteriormente haremos un markowitz de 100 carteras con pesos distintos y seleccionaremos aquella cartera que nos de un mayor sharpe. De este modo, favorecemos la exploración. 

**2. El método de selección de padres no parece, a priori, problemático, pero el cruce y la mutación sí. ¿Cómo va a heredar el hijo la información genética de los padres? Aquí tenemos un triple problema a resolver:**

**2.1 ¿Cual es el número de activos que hereda un hijo de dos padres? Si un padre tiene 5 activos y el otro 7 (distintos entre sí), ¿cuantos activos tendrá el hijo? Debemos pensar en el punto primero. El hijo no puede tener un número de activos creciente en cada generación. El sistema tiene que ser dinámico, creciente y decreciente, respetando los límites de 1 y 20.**

Para la herencia de los padres a los hijos, lo vamos a solucionar de distintas formas. En primer lugar, vamos a eliminar los activos duplicados y luego vamos a generar una matriz de correlaciones. Dentro de la matriz de correlaciones calcularemos la correlación media de todos los activos, y luego para cada activo estableceremos un umbral. Si supera ese umbral, diremos que el activo está correlacionado con el resto, mientras que si no supera ese umbral diremos que el activo está descorrelacionado. Los activos descorrelacionados los va a heredar el hijo, mientras que de los activos más correlacionados simplemente nos quedaremos con aquél que tenga el mejor sharpe. Existirán casos en los que todos los activos aparezcan descorrelados y otros en los que aparezcan todos correlacionados. 

En el caso de que todos sean descorrelacionados, si están dentro del rango de 20 activos, los heredará todos; mientras que si supera los 20 activos, entonces los ordenaremos de mayor a menor sharpe, y haremos que seleccione en ese mismo orden, entre 10 y 20 activos (el número de activos será aleatorio). En el caso de que estén todos correlacionados, simplemente nos quedaremos con uno, aquél que tenga el mayor sharpe. Para que esto tenga sentido, estableceremos un método de mutación, explicado más adelante. 

**2.2 ¿Qué porcentaje de inversión hereda cada activo para que sume 100%? Debemos permitir la construcción de carteras donde unos pocos activos (1 o 2) se lleven un porcentaje muy elevado del capital disponible, para explorar todas las soluciones. Reescalar los pesos no es una solución aceptable.**

Los hijos no heredarán ningún peso, simplemenete heredarán activos. Para cada hijo le pasaremos una función encargada de realizar un markowitz con 100 carteras con pesos distintos, y seleccionaremos aquellos pesos que nos den un mayor sharpe. De este modo favorecemos la exploración e intentamos que la generación mejore la anterior. 

**2.3 La evolución, en este problema no puede limitarse a selección y cruzamiento o caeremos rápidamente en un mínimo local. Debemos encontrar un sistema que equilibre la selección con la exploración, si queremos encontrar la cartera óptima.** 

Para favorecer la exploración, vamos a realizar un random entre 1 y 4 activos que se van a añadir a la cartera heredada por cada hijo, de forma que siempre se favorezca la exploración y no caigamos en un mínimo local.

In [1]:
import pandas as pd
import numpy as np
import random
import time
import pickle
from time import time
from timeit import timeit

In [2]:
with open('fondos_limpios.pickle', 'rb') as handle:
    fondos_limpios = pickle.load(handle)
returns = np.log(fondos_limpios).diff().dropna()
if any(returns.sum() <= 0):
    fondos_limpios = fondos_limpios.drop(fondos_limpios.columns[np.where(returns.sum() <= 0)], axis = 1)

In [3]:
class generaciones:
    
    def __init__(self, numero_individuos, n_generaciones, fondos_limpios):
        self.numero_individuos = numero_individuos
        self.n_generaciones = n_generaciones
        self.fondos_limpios = fondos_limpios
    
    def selector_activos(self):
        activos_seleccionados = [np.random.randint(0, self.fondos_limpios.shape[1], size = np.random.randint(2,20)) for n in range(0,self.numero_individuos)]
        return(activos_seleccionados)
          
    def calculo_eficiencia(self, activos_seleccionados):
        
        eficiencia = []
        weights = []
        for seleccion in activos_seleccionados:         
            returns = np.log(self.fondos_limpios.iloc[:,seleccion]).diff().dropna()
            pesos= np.random.uniform(0,100,(100,len(seleccion)))
            pesos = pd.DataFrame(pesos)
            pesos = pesos.div(np.sum(pesos, axis=1), axis=0)
            pesos = np.array(pesos.transpose())
            matriz_covarianzas = returns.cov()
            rentabilidad_diaria = np.mean(returns, axis=0).values
            rentabilidad_cartera = np.dot(pesos.T,rentabilidad_diaria)
            riesgo_carteras = np.dot(pesos.T,np.sum(np.dot(matriz_covarianzas,pesos), axis = 1))
            if any(riesgo_carteras <= 0):
                eliminar = np.where(riesgo_carteras <= 0)
                riesgo_carteras = np.delete(riesgo_carteras,eliminar)
                rentabilidad_cartera = np.delete(rentabilidad_cartera,eliminar)
            riesgo_carteras = np.sqrt(riesgo_carteras)
            eficiencia_carteras = rentabilidad_cartera/riesgo_carteras
            eficiencia.append(eficiencia_carteras[np.argmax(eficiencia_carteras)]) 
            weights.append(pesos[:,np.argmax(eficiencia_carteras)])
        return(eficiencia, weights)       
    
    def selector_padres(self, eficiencia):
        
        maximo = np.max(eficiencia)
        minimo = np.min(eficiencia)
        valores_fitness = [(valor-minimo)/(maximo-minimo) for valor in eficiencia]
        seleccionados = 0
        padres = []
        while seleccionados < ((self.numero_individuos-1)*2):
            candidato_papa = random.randint(0,(self.numero_individuos-1))
            umbral = random.uniform(0,1)
            if valores_fitness[candidato_papa] < umbral:
                if len(padres) == 0:
                    padres.append(candidato_papa)
                    seleccionados = seleccionados + 1 
                else:
                    if candidato_papa != padres[-1]:
                        padres.append(candidato_papa)
                        seleccionados = seleccionados + 1                
            else:
                continue 
                
        padres_madres = []
        for i in range(0,(((self.numero_individuos-1)*2)+1),2):
            if i == 0:
                anterior = i
            else:
                tupla = tuple(padres[anterior:i])
                anterior = i 
                padres_madres.append(tupla)

        return(padres_madres)
    
    def nuevas_generaciones(self, padres, activos_seleccionados, eficiencia):
    
        nueva_generacion = []

        #Metemos al mejor padre
        nueva_generacion.append(list(activos_seleccionados[np.argmax(eficiencia)]))

        for familia in padres:

            papa = activos_seleccionados[familia[0]]
            mama = activos_seleccionados[familia[1]]
            #juntamos la herencia de papa y mama, y eliminamos los activos duplicados
            herencia = list(set(np.concatenate((papa,mama)))) 

            #calculamos sharpe de los activos y clusterizamos por correlacion
            rentabilidad = (np.log(self.fondos_limpios.iloc[:,herencia]).diff().dropna())
            sharpe = rentabilidad.mean()/rentabilidad.var()
            sharpe.index = herencia
            matriz_corr = rentabilidad.corr()
            corr_media = np.mean(matriz_corr.sum()/matriz_corr.shape[0])
            umbral_rechazo = matriz_corr.shape[0] * 0.6
            activos_correlados = []
            activos_descorrelados = []
            for activo in range(matriz_corr.shape[0]):
                corr_encima_media = (matriz_corr.iloc[0] >= corr_media).value_counts()[True]
                if corr_encima_media >= umbral_rechazo:
                    activos_correlados.append(activo)
                else:
                    activos_descorrelados.append(activo)
        
            if len(activos_correlados) > 1: 
                
                mejor_sharpe = np.argmax(sharpe.iloc[activos_correlados])
                if len(activos_descorrelados) == 0:
                    genes_heredados = [herencia[mejor_sharpe]]
                else:
                    genes_heredados = list(np.array(herencia)[list(np.append(activos_descorrelados, mejor_sharpe))])

            else:    
                genes_heredados = list(np.array(herencia)[activos_descorrelados])
            
            if len(genes_heredados) > 20:
                genes_heredados = list(sharpe.iloc[activos_descorrelados].sort_values(ascending = False)[:np.random.randint(10,20)].index)
            
            n_activos_heredar = len(genes_heredados)
            
            if n_activos_heredar < 16:
                mutacion = n_activos_heredar + np.random.randint(1,5)
                exploracion = mutacion - n_activos_heredar
                genes_mutados = [np.random.randint(0, fondos_limpios.shape[1]) for n in range(exploracion)]
                genes_hijo = list(set(np.concatenate((genes_heredados, genes_mutados))))
                if mutacion != len(genes_hijo):
                    while mutacion != len(genes_hijo):
                        genes_mutados = [np.random.randint(0, fondos_limpios.shape[1]) for n in range(exploracion)]
                        genes_hijo = list(set(np.concatenate((genes_heredados, genes_mutados))))
                nueva_generacion.append(genes_hijo)
            else:
                nueva_generacion.append(genes_heredados)

        return(nueva_generacion)
    
    def funcion_definitiva(self):
        
        eficiencia_generacional = []
        activos_seleccionados = self.selector_activos()
        eficiencia, pesos = self.calculo_eficiencia(activos_seleccionados) 
        
        padres = self.selector_padres(eficiencia)
        
        
        mejor_sharpe = eficiencia[np.argmax(eficiencia)]
        cartera_optima = {'Sharpe': mejor_sharpe, 'activos': list(fondos_limpios.columns[activos_seleccionados[np.argmax(eficiencia)]]), 'pesos': pesos[np.argmax(eficiencia)]}
        
        eficiencia_generacional.append(mejor_sharpe)
        
        for generaciones in range(self.n_generaciones):
            nueva_generacion = self.nuevas_generaciones(padres, activos_seleccionados, eficiencia)
            eficiencia, pesos = self.calculo_eficiencia(nueva_generacion)
            eficiencia[0] = mejor_sharpe #no dejamos que el cálculo de eficiencia nos cambie los pesos al mejor padre de la generación anterior
            
            padres = self.selector_padres(eficiencia)
            mejor_sharpe = eficiencia[np.argmax(eficiencia)]
            cartera_optima = {'Sharpe': mejor_sharpe, 'activos': list(fondos_limpios.columns[activos_seleccionados[np.argmax(eficiencia)]]), 'pesos': pesos[np.argmax(eficiencia)]}
            activos_seleccionados = [np.array(nueva_generacion[item]) for item in range(len(nueva_generacion))]
            eficiencia_generacional.append(mejor_sharpe)
            
                
        resultado = eficiencia_generacional
                
        return(resultado, cartera_optima)
        

In [6]:
tiempo_inicial = time()

numero_individuos = 50
n_generaciones = 50
resultado, cartera_optima = generaciones(numero_individuos, n_generaciones, fondos_limpios).funcion_definitiva()
tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial
print ('El tiempo de ejecucion fue:',tiempo_ejecucion )    
print(cartera_optima)

El tiempo de ejecucion fue: 37.81190752983093
{'Sharpe': 0.14827761067324943, 'activos': ['LU0568621618', 'IE0004807107', 'GB0031953234', 'LU0350842315', 'LU0933169087', 'LU1873130071', 'LU0284585170', 'LU1624421126', 'IE00B3L10240', 'IE00BPT2BJ75', 'IE00B45H7020', 'IE00B3L10687'], 'pesos': array([0.07083155, 0.04625315, 0.01930084, 0.02578767, 0.16989489,
       0.0205318 , 0.04504189, 0.17940914, 0.0805752 , 0.0053477 ,
       0.15597182, 0.18105434])}


In [7]:
resultado

[0.020197827629495485,
 0.020197827629495485,
 0.020197827629495485,
 0.020197827629495485,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.07175737732641499,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.08548155987259182,
 0.09919946144497266,
 0.09919946144497266,
 0.12517983663510102,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.13651236173322098,
 0.14827761067324943,
 0.14827761067324943,
 0.14827761067324943,
 0.14827761067324943,
 0.14827761067324943,
 0.14827761067324943,
 0.148